<a name="outline"></a>

## Outline

- [Task 1](#task-1): Regression
  - [1.1](#q11) Linear regression
  - [1.2](#q12) Ridge regression
  - [1.3](#q13) Relaxation of Lasso regression
- [Task 2](#task-2): Classification
  - [2.1](#q21) kNN classifier
  - [2.2](#q22) Random forest 
  - [2.3](#q23) Support vector machine (SVM) 
- [Task 3](#task-3): Mastery component 
  - [3.1](#q31) Logistic regression and bagging 
  - [3.2](#q32) Kernelised SVM classifier 

<a name="task-1"></a>
# Task 1: Regression [^](#outline)



<a name="q11"></a>

## 1.1  [^](#outline)



1.1.1

I have extracted the data from csv file 'chemistry_samples' using pandas. I have then analysed the data to understand the format in which it is presented. This is important if the data contains any date columns as they will need to be parsed, but also for calculations, to know which columns can be used as numerical and others and categorical. I do this through firstly looking at the first 5 rows of the data. This will most likely give me most of the information I need to understand the data given, however there could be other rows further in the data that don't match the same as the first 5 and such can perform some more analysis to make sure our first glance is correct. I then run some analysis on the actual data provided to find the data types of the values. This allows me to see if the pattern I had spotted earlier was true, and from the output we can see that all the columns are all floating point values, and such can be treated numerically. This also confirms that no where within the data we have mistakes with letters creaping into the data which would need to be filtered to allow for the regression to continue. I have also calculated some basic statistics for each of the columns values. This is important to give me  quick summary of the data given. This allows me to see if there are any anomalies within the data, and also gives me the number of entries for each of the columns, which are all equal to one another, and with another analysis on the line below showing the shape of the data, confirms that every row is filled with all the variables, and such is a complete data set.

Then I implemented standard linear regression using just the variables given. This gives me 10 beta values whcih correspond to the variables given, to predict the LC50 value.  Using these values I then test this against the training data to get an $R^2$ score, and display the beta coefficients found. I have printed this below. The $R^2$ score is a statistical measure for a regression model that represents the proportion of variance in the variables given that can be explained by the LC50 variable. This means that a higher $R^2$ score represents a better fitted model to the data. We can see that it returns an $R^2$ score of 0.77, which means that the regression found is quite a good fit, with a score of 0 being a horizontal line through the mean of the LC50 variable.

I then implemented the augmented linear regression with an added intercept parameter. I do this as the data may not be normed to go exactly through the orgin, but the first regression method I implemented is limited to having to go through the origin. So by adding this intercept parameter, it enables the augmented linear regression to improve from the original regression, or if the data is normed, remain the same. By adding this intercept it solves the problem of data not normed through the origin, and as such I implemented the augmented linear regression below and compared the results. I calculated the $R^2$ score and displayed the augmented beta parameters found. They are presented below. This time the $R^2$ score is 0.87 which means that for the training data, this is a much better fit than the original regression implemented. However, this will need to be confirmed with unseen test data as it could be overfitting the training data and not fit the global pattern. However, this improved $R^2$ score will be the cause that the Intercept parameter added is not equal to 0, and such suggests that target data is not normalised and such the augmented linear regression is better suited for this problem. I have also compared the two methods in a data frame at the end, which allows me to see how the two methods differ in importance of the variables when the intercept parameter is added.


In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [105]:
# load data as pandas data frames
chemistry_samples = pd.read_csv('chemistry_samples.csv')

In [106]:
chemistry_samples.head()

,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,FV1,VFV,FV2,FV3,LC50
0,3.661280,0.658363,1.602232,1.994272,0.836488,3.153623,15.893033,-27.724370,0.059355,0.756698,5.506249
1,3.936362,1.154287,1.146997,0.904295,2.948308,5.141095,13.590177,-31.821521,-13.408855,1.161298,6.636791
2,0.964144,0.415485,1.481028,2.136585,0.043679,-1.156783,15.989419,-3.699312,2.561525,0.500115,1.563388
3,2.074617,1.417296,0.486216,0.000908,-0.066980,2.610960,7.962046,-16.374439,2.448975,1.481888,6.248432
4,1.448569,0.836892,1.951012,0.028318,-0.039121,1.851095,22.285266,-9.526361,2.870400,0.649234,3.676796


In [107]:
chemistry_samples.dtypes

CIC0         float64
SM1_Dz(Z)    float64
GATS1i       float64
NdsCH        float64
NdssC        float64
MLOGP        float64
FV1          float64
VFV          float64
FV2          float64
FV3          float64
LC50         float64
dtype: object

In [108]:
chemistry_samples.describe()

,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,FV1,VFV,FV2,FV3,LC50
count,4111.000000,4111.000000,4111.000000,4111.000000,4111.000000,4111.000000,4111.000000,4111.000000,4111.000000,4111.000000,4111.000000
mean,2.907930,0.635057,1.295748,0.449253,0.540840,2.093087,15.057563,-22.111122,0.302976,0.637047,4.139930
std,0.782757,0.427101,0.396828,0.697176,0.940516,1.447468,3.539621,7.110208,5.239923,0.459752,1.206448
min,0.508140,0.000209,0.289772,0.000158,-0.600682,-3.505785,4.715537,-50.284071,-35.369667,-0.162048,0.124518
25%,2.324115,0.286364,0.965427,0.054057,-0.056750,1.141446,12.342002,-26.996891,-1.033571,0.285723,3.348342
50%,2.918325,0.560186,1.261636,0.122348,0.116859,2.064408,14.707910,-22.326363,2.361851,0.536016,4.083925
75%,3.438485,0.900059,1.556928,0.860789,0.978137,3.022278,17.391135,-16.808405,3.287077,0.890875,4.901910
max,6.286216,2.221232,2.950545,4.295030,6.328623,6.845000,31.153722,0.038877,5.981860,3.447559,8.751823


In [109]:
chemistry_samples.shape

(4111, 11)

In [167]:
def max_lik_estimate(X, y):
    
    # X: N x D matrix of training inputs
    # y: N x 1 vector of training targets/observations
    # returns: maximum likelihood parameters (D x 1)
    
    N, D = X.shape
    beta_ml = np.linalg.solve(X.T @ X, X.T @ y)
    return beta_ml


def predict_with_estimate(X_test, beta):
    
    # X_test: K x D matrix of test inputs
    # beta: D x 1 vector of parameters
    # returns: prediction of f(X_test); K x 1 vector
    
    prediction = X_test @ beta
    
    return prediction 

In [168]:
# define the train set
chem_train = chemistry_samples.iloc[:, :-1].values

# define the train target values
chem_train_y = chemistry_samples.iloc[:, -1].values

# work out the maximum likelihood estimates
chem_train_beta_ml = max_lik_estimate(chem_train, chem_train_y)

# predict the function values at the test points using the maximum likelihood estimator
chem_train_ml_prediction_y = predict_with_estimate(chem_train, chem_train_beta_ml)

In [169]:
def r2_score(y_test, y_pred):
    
    # y_test: N x 1 vector of actual values
    # y_pred: N x 1 vector of predicted values
    # returns: r2 score of the predictions

    numerator = np.sum((y_test-y_pred)**2)
    y_avg = np.mean(y_test)
    denominator = np.sum((y_test-y_avg)**2)
    
    return 1 - numerator/denominator

In [170]:
print(r'R2 score:', r2_score(chem_train_y, chem_train_ml_prediction_y))

R2 score: 0.7712119635266652


In [171]:
[(j, chem_train_beta_ml[i]) for i, j in enumerate(chemistry_samples.columns[:-1].values)]

[('CIC0', 0.6203733928550708),
 ('SM1_Dz(Z)', 1.6806804178200736),
 ('GATS1i', -0.5661420478588721),
 ('NdsCH', 0.41624127447550235),
 ('NdssC', 0.06372340713960672),
 ('MLOGP', 0.430638221079959),
 ('FV1', 0.05704399647373448),
 ('VFV', 0.0012511248583359053),
 ('FV2', 0.01427984772537641),
 ('FV3', -0.01916889608754055)]

In [172]:
def max_lik_estimate_aug(X_aug, y):
    
    # X_aug: N x D+1 matrix of training inputs
    # y: N x 1 vector of training targets/observations
    # returns: maximum likelihood parameters (D+1 x 1)
    
    beta_aug_ml = max_lik_estimate(X_aug, y)
    
    return beta_aug_ml

In [173]:
# define the train set
chem_train = chemistry_samples.iloc[:, :-1].values

# define the augmented train set 
chem_train_aug = np.hstack([np.ones((chem_train.shape[0],1)), chem_train])

# define the train target values
chem_train_y = chemistry_samples.iloc[:, -1].values

# work out the augmented maximum likelihood estimates
chem_train_aug_beta_ml = max_lik_estimate_aug(chem_train_aug, chem_train_y)

# predict the function values at the test points using the augmented maximum likelihood estimator
chem_train_aug_ml_prediction_y = predict_with_estimate(chem_train_aug, chem_train_aug_beta_ml)

In [174]:
print(r'R2 score:', r2_score(chem_train_y, chem_train_aug_ml_prediction_y))

R2 score: 0.8718259975718016


In [175]:
[(j, chem_train_aug_beta_ml[i]) for i, j in enumerate(np.concatenate((np.array(['Intercept']), chemistry_samples.columns[:-1].values)))]

[('Intercept', 2.6163804133558006),
 ('CIC0', 0.044713833314312355),
 ('SM1_Dz(Z)', 1.2587188417698731),
 ('GATS1i', -0.03800927663571632),
 ('NdsCH', 0.3630734478276748),
 ('NdssC', 0.004665348846348374),
 ('MLOGP', 0.39051005215303064),
 ('FV1', -0.07460286288002355),
 ('VFV', -0.035706946023729334),
 ('FV2', -0.015258825830479272),
 ('FV3', -0.0018031593979083939)]

In [179]:
headers = ['Intercept'] + list(chemistry_samples.columns[:-1].values) + ['R2 score']
data = [[0] + list(chem_train_beta_ml) + [r2_score(chem_train_y, chem_train_ml_prediction_y)],
list(chem_train_aug_beta_ml) + [r2_score(chem_train_y, chem_train_aug_ml_prediction_y)]]
rows = ['linear regression', 'augmented linear regression']
pd.DataFrame(data, index=rows, columns=headers)

,Intercept,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,FV1,VFV,FV2,FV3,R2 score
linear regression,0.00000,0.620373,1.680680,-0.566142,0.416241,0.063723,0.430638,0.057044,0.001251,0.014280,-0.019169,0.771212
augmented linear regression,2.61638,0.044714,1.258719,-0.038009,0.363073,0.004665,0.390510,-0.074603,-0.035707,-0.015259,-0.001803,0.871826


1.1.2

I have extracted the data from the csv file 'chemistry_test' and carried out the same analysis as the first file above. This confirms the same pattern as before and therefore allows me to carry on with my regression with no need to filtering. I then extract the test parameters to use, along with the test values which will be used to compare the estimates to their actual values. I start by using the original linear regression on the test parameters, using the calculated beta coefficients from the previous part, and then work out its $R^2$ score. The $R^2$ score is 0.76, which is within a variance of the test data, meaning that it is close enough to signify that the model performs at the consistent level for any prediction made. This is because it is a very similar score to its previous score, meaning that the regression is not overfit to the training data and should give global predictions at the same score.

I then used the augmented linear regression model to predict the unseen test data and this returned an $R^2$ score of 0.86 which is also very close to its score on the training data. This means that it also has a consistent fit that isn't overfit to the training data. But I also notice that the $R^2$ score for the augmented linear regression is higher on both the in and out of sample data, which indicates that it is a much better fit that the original linear regression model. This would also confirm that the data is not normalised through the origin and such any other models we will fit to the data should include this intercept value also. I have presented the scores in a table below.

In [51]:
# load data as pandas data frames
chemistry_test = pd.read_csv('chemistry_test.csv')

In [52]:
chemistry_test.head()

,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,FV1,VFV,FV2,FV3,LC50
0,1.617579,0.376167,1.224281,0.849464,1.101738,-0.448372,14.913614,-9.091450,-1.953849,0.328298,1.791786
1,4.278766,0.514788,1.259734,0.210436,0.819626,4.446118,12.904817,-37.986185,-2.804426,0.452758,6.125609
2,3.578652,0.221018,1.552583,1.007153,-0.013073,1.960720,17.393050,-27.188863,3.565159,0.341665,3.953270
3,3.021762,1.602774,1.044233,0.054776,2.060890,4.510903,12.777434,-22.710306,-7.966119,1.729511,6.995314
4,2.487163,0.799948,1.005727,0.094923,0.146542,2.298082,13.336721,-16.839870,2.607198,0.904353,5.253633


In [119]:
chemistry_test.dtypes

CIC0         float64
SM1_Dz(Z)    float64
GATS1i       float64
NdsCH        float64
NdssC        float64
MLOGP        float64
FV1          float64
VFV          float64
FV2          float64
FV3          float64
LC50         float64
dtype: object

In [121]:
chemistry_test.describe()

,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,FV1,VFV,FV2,FV3,LC50
count,1028.000000,1028.000000,1028.000000,1028.000000,1028.000000,1028.000000,1028.000000,1028.000000,1028.000000,1028.000000,1028.000000
mean,2.861501,0.631125,1.269849,0.455870,0.494666,2.068875,14.818163,-21.742714,0.584122,0.632418,4.120735
std,0.774397,0.418012,0.382645,0.719690,0.911657,1.450723,3.494455,7.160118,4.700937,0.450990,1.175629
min,0.647476,0.003592,0.202975,0.000323,-0.540236,-2.441058,5.395300,-49.146675,-26.198937,-0.195405,1.161957
25%,2.273664,0.286499,0.967198,0.053715,-0.068329,1.159844,12.140332,-26.549683,-0.594728,0.271490,3.303218
50%,2.897980,0.554753,1.222545,0.124618,0.108270,2.088124,14.471913,-21.912708,2.503755,0.533540,4.110352
75%,3.411725,0.903278,1.523901,0.871427,0.932910,2.978749,16.934890,-16.346515,3.353090,0.898143,4.822600
max,5.801678,2.013109,2.699048,4.342691,6.129559,6.679374,28.259679,0.681013,5.618493,2.105268,8.068737


In [122]:
chemistry_test.shape

(1028, 11)

In [123]:
# define the test set
chem_test = chemistry_test.iloc[:, :-1].values

# define the test target values
chem_test_y = chemistry_test.iloc[:, -1].values

# predict the function values at the test points using the maximum likelihood estimator
chem_test_ml_prediction_y = predict_with_estimate(chem_test, chem_train_beta_ml)

In [124]:
print(r'R2 score:', r2_score(chem_test_y, chem_test_ml_prediction_y))

R2 score: 0.7617663939067444


In [125]:
# define the augmented test set 
chem_test_aug = np.hstack([np.ones((chem_test.shape[0],1)), chem_test])

# predict the function values at the test points using the augmented maximum likelihood estimator
chem_test_aug_ml_prediction_y = predict_with_estimate(chem_test_aug, chem_train_aug_beta_ml)

In [126]:
print(r'R2 score:', r2_score(chem_test_y, chem_test_aug_ml_prediction_y))

R2 score: 0.8642933369927278


In [180]:
headers = ['In-sample R2 score', 'Out-of-sample R2 score']
data = [[r2_score(chem_train_y, chem_train_ml_prediction_y), r2_score(chem_test_y, chem_test_ml_prediction_y)],
[r2_score(chem_train_y, chem_train_aug_ml_prediction_y), r2_score(chem_test_y, chem_test_aug_ml_prediction_y)]]
rows = ['linear regression', 'augmented linear regression']
pd.DataFrame(data, index=rows, columns=headers)

,In-sample R2 score,Out-of-sample R2 score
linear regression,0.771212,0.761766
augmented linear regression,0.871826,0.864293


<a name="q12"></a>
## 1.2 [^](#outline)

1.2.1

In [241]:
def ridge_estimate(X, y, penalty):
    
    # X: N x D matrix of training inputs
    # y: N x 1 vector of training targets/observations
    # returns: maximum likelihood parameters (D x 1)
    
    N, D = X.shape
    X_aug = np.hstack([np.ones((N,1)), X]) # augmented training inputs of size N x (D+1)
    N_aug, D_aug = X_aug.shape
    I = np.identity(D_aug)
    #I[0,0] = 0
    beta_ridge = np.linalg.solve(X_aug.T @ X_aug + penalty * I, X_aug.T @ y).reshape(-1,1)
    return beta_ridge

In [244]:
# define the train set
chem_train = chemistry_samples.iloc[:, :-1].values

# define the augmented test set 
chem_train_aug = np.hstack([np.ones((chem_train.shape[0],1)), chem_train])

# define the train target values
chem_train_y = chemistry_samples.iloc[:, -1].values

# set penalty
penalty = 1

# work out the augmented maximum likelihood estimates
chem_train_beta_ridge = ridge_estimate(chem_train, chem_train_y, penalty)

# predict the function values at the test points using the augmented maximum likelihood estimator
chem_train_ridge_prediction_y = predict_with_estimate(chem_train_aug, chem_train_beta_ridge)

In [245]:
print(r'R2 score:', r2_score(chem_train_y, chem_train_ridge_prediction_y))

R2 score: -7696.973849818741


In [211]:
def score_ridge_MSE(X_train, y_train, X_test, y_test, penalty):
    
    # X_train: 4M x D matrix of training inputs
    # y_train: 4M x 1 vector of training targets
    # X_test: M x D matrix of test inputs
    # y_test: M x 1 vector of test targets
    # returns: MSE of the ridge prediction
    
    beta_aug = ridge_estimate(X_train, y_train, penalty)
    X_test_aug = np.hstack([np.ones((X_test.shape[0],1)), X_test])
    y_pred = predict_with_estimate(X_test_aug, beta_aug)
    return np.mean((y_test-y_pred)**2)


def cross_validation_score_ridge(X_train, y_train, folds, penalty):
    
    # X_train: N x D matrix of training inputs
    # y_train: N x 1 vector of training targets
    # folds: f x N/f matrix of indexes to cross validate
    # penalty: float of the penalty for ridge regression
    # returns: average MSE score for the given penalty ridge regression
    
    scores = []
    for i in range(len(folds)):
        val_indexes = folds[i]
        train_indexes = list(set(range(y_train.shape[0])) - set(val_indexes))
    
        X_train_i = X_train[train_indexes, :]
        y_train_i = y_train[train_indexes]

        X_val_i = X_train[val_indexes, :]
        y_val_i = y_train[val_indexes]

        score_i = score_ridge_MSE(X_train_i, y_train_i, X_val_i, y_val_i, penalty)
        scores.append(score_i)
    
    # Return the average score
    return sum(scores) / len(scores)


def choose_best_penalty(X_train, y_train, folds, penalty_range):
    
    # X_train: N x D matrix of training inputs
    # y_train: N x 1 vector of training targets
    # folds: f x N/f matrix of indexes to cross validate
    # penalty_range: M x 1 vector of penalty values to test
    # returns: lowest average MSE penalty value from penalty range
    
    penalty_scores = np.zeros((len(penalty_range),))

    for i, penalty in enumerate(penalty_range):
        penalty_scores[i] = cross_validation_score_ridge(X_train, y_train, folds, penalty)
        print(f'CV_ACC@penalty={penalty}: {penalty_scores[i]:.3f}')

    best_penalty_index = np.argmin(penalty_scores)
    return penalty_range[best_penalty_index]


def define_folds_indexes(y_test, folds):
    
    # y_test: N x 1 vector of training targets
    # folds: integer of the number of folds wanted
    # returns: the folds indexes for the parameters given
    
    return np.split(np.arange(len(y_test)), [int(x*len(y_test)/folds) for x in range(1, folds)])


In [212]:
# define the train set
chem_train = chemistry_samples.iloc[:, :-1].values

# define the train target values
chem_train_y = chemistry_samples.iloc[:, -1].values

# choose the number of folds wanted, in the question it is 5
folds = 5

# define the fold indexes to be used in the cross validation
folds_indexes = define_folds_indexes(chem_train_y, folds)

# define the penalty range to test
penalty_range = np.linspace(0, 1, 20)

# work out the best penalty
best_penalty = choose_best_penalty(chem_train, chem_train_y, folds_indexes, penalty_range)
print('best_penalty:', best_penalty)


CV_ACC@penalty=0.0: 2.723
CV_ACC@penalty=0.05263157894736842: 2.723
CV_ACC@penalty=0.10526315789473684: 2.723
CV_ACC@penalty=0.15789473684210525: 2.723
CV_ACC@penalty=0.21052631578947367: 2.723
CV_ACC@penalty=0.2631578947368421: 2.723
CV_ACC@penalty=0.3157894736842105: 2.723
CV_ACC@penalty=0.3684210526315789: 2.723
CV_ACC@penalty=0.42105263157894735: 2.723
CV_ACC@penalty=0.47368421052631576: 2.723
CV_ACC@penalty=0.5263157894736842: 2.723
CV_ACC@penalty=0.5789473684210527: 2.723
CV_ACC@penalty=0.631578947368421: 2.723
CV_ACC@penalty=0.6842105263157894: 2.723
CV_ACC@penalty=0.7368421052631579: 2.723
CV_ACC@penalty=0.7894736842105263: 2.723
CV_ACC@penalty=0.8421052631578947: 2.723
CV_ACC@penalty=0.894736842105263: 2.722
CV_ACC@penalty=0.9473684210526315: 2.722
CV_ACC@penalty=1.0: 2.722
best_penalty: 1.0


<a name="q13"></a>

## 1.3 [^](#outline)




In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def huber(beta, c=1e-6):
    return None  # <-- EDIT THIS LINE
  
def grad_huber(beta, c=1e-6):
    return None  # <-- EDIT THIS LINE

In [ ]:
def minimize_ls_huber(X, y, lambd, n_iters=10000, step_size=1e-3):
    n, p = X.shape
    XX = X.T @ X / n
    Xy = X.T @ y / n
    
    # next line: initialise betas
    beta = np.zeros(shape=(p, 1))

    # gradient descent (hint: see logistic regression coding task)
    for i in range(n_iters):
        grad = None  # <-- EDIT THIS LINE
        # next line: gradient descent update
        beta = None  # <-- EDIT THIS LINE
      
    return beta

<a name="task-2"></a>

# Task 2: Classification [^](#outline)

<a name="q21"></a>

## 2.1 [^](#outline)

<a name="q22"></a>

## 2.2 [^](#outline)

<a name="q23"></a>

## 2.3 [^](#outline)

<a name="task-3"></a>

# Task 3: Mastery Component [^](#outline)

<a name="q31"></a>

## 3.1 [^](#outline)

<a name="q32"></a>

## 3.2 [^](#outline)